In [19]:
from PIL import Image
import numpy as np

data = np.array([[10, 50, 200], [30, 127, 210], [80, 190, 255]], dtype=np.uint8)
img_pil = Image.fromarray(data, mode='L')  

print(type(img_pil))  # <class 'PIL.Image.Image'>
print(np.array(img_pil))

<class 'PIL.Image.Image'>
[[ 10  50 200]
 [ 30 127 210]
 [ 80 190 255]]


In [29]:
import cv2

def apply_clahe(img):
    img_np = np.array(img)  # Конвертация в numpy (3x3)
    print("До CLAHE:\n", img_np[:3, :3])  # Первые 3x3 пикселя
    
    clahe = cv2.createCLAHE(clipLimit=0.05, tileGridSize=(2, 2))
    img_clahe = clahe.apply(img_np)
    print("После CLAHE:\n", img_clahe[:3, :3])
    
    return Image.fromarray(img_clahe)

apply_clahe(img_pil)

До CLAHE:
 [[ 10  50 200]
 [ 30 127 210]
 [ 80 190 255]]
После CLAHE:
 [[ 64 191 223]
 [128 255 255]
 [160 255 255]]


In [31]:
mask = [[1, 0, 0],
 [255, 255, 0],
 [254, 255, 0]]

mask = np.array(mask)  # Исходная маска (0 и 255)
print("Маска до бинаризации:\n", mask[:3, :3])

mask = (mask > 127).astype(np.uint8)  # Порог 127
print("Маска после бинаризации:\n", mask[:3, :3])

# Проверка нормализации
normalized = (data - 127.5) / 127.5
print("До:\n", data)
print("После:\n", normalized)

# Обратное преобразование
restored = (normalized * 127.5 + 127.5).astype(np.uint8)
print("Восстановлено:\n", restored)

Маска до бинаризации:
 [[  1   0   0]
 [255 255   0]
 [254 255   0]]
Маска после бинаризации:
 [[0 0 0]
 [1 1 0]
 [1 1 0]]
До:
 [[ 10  50 200]
 [ 30 127 210]
 [ 80 190 255]]
После:
 [[-0.92156863 -0.60784314  0.56862745]
 [-0.76470588 -0.00392157  0.64705882]
 [-0.37254902  0.49019608  1.        ]]
Восстановлено:
 [[ 10  50 200]
 [ 30 127 210]
 [ 80 190 255]]


In [2]:
import torch
import torch.nn as nn
from torchvision.models import resnet34

# 1. Инициализация модели
encoder = resnet34(weights=None)  # Используем новый синтаксис torchvision
encoder.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# 2. Функция для логирования
def log_shape(name, tensor):
    print(f"{name}: {tuple(tensor.shape)}")

# 3. Тестовый тензор (1 изображение 512x512 grayscale)
x = torch.randn(1, 1, 512, 512)
print("\n=== Детальный проход через энкодер ===")
log_shape("Вход", x)

# 4. Вручную проходим все слои
with torch.no_grad():
    # Conv1
    x = encoder.conv1(x)
    log_shape("После conv1 (7x7, stride=2)", x)
    
    # BatchNorm + ReLU
    x = encoder.bn1(x)
    x = encoder.relu(x)
    log_shape("После bn1 + relu", x)
    
    # MaxPool
    x = encoder.maxpool(x)
    log_shape("После maxpool (3x3, stride=2)", x)
    
    # Детализация Layer1 (3 блока)
    print("\n--- Layer1 ---")
    for i, block in enumerate(encoder.layer1):
        x_prev = x.clone()
        x = block(x)
        log_shape(f"Block {i+1} input", x_prev)
        log_shape(f"Block {i+1} output", x)
        print(f"Downsample: {hasattr(block, 'downsample')}")
    
    # Детализация Layer2 (4 блока)
    print("\n--- Layer2 ---")
    for i, block in enumerate(encoder.layer2):
        x_prev = x.clone()
        x = block(x)
        log_shape(f"Block {i+1} input", x_prev)
        log_shape(f"Block {i+1} output", x)
        print(f"Downsample: {hasattr(block, 'downsample')}")
    
    # Аналогично для Layer3 и Layer4
    print("\n--- Layer3 ---")
    for i, block in enumerate(encoder.layer3):
        x_prev = x.clone()
        x = block(x)
        log_shape(f"Block {i+1} input", x_prev)
        log_shape(f"Block {i+1} output", x)
        print(f"Downsample: {hasattr(block, 'downsample')}")
    
    print("\n--- Layer4 ---")
    for i, block in enumerate(encoder.layer4):
        x_prev = x.clone()
        x = block(x)
        log_shape(f"Block {i+1} input", x_prev)
        log_shape(f"Block {i+1} output", x)
        print(f"Downsample: {hasattr(block, 'downsample')}")

print("\n=== Итоговый выход энкодера ===")
log_shape("Финальный тензор", x)


=== Детальный проход через энкодер ===
Вход: (1, 1, 512, 512)
После conv1 (7x7, stride=2): (1, 64, 256, 256)
После bn1 + relu: (1, 64, 256, 256)
После maxpool (3x3, stride=2): (1, 64, 128, 128)

--- Layer1 ---
Block 1 input: (1, 64, 128, 128)
Block 1 output: (1, 64, 128, 128)
Downsample: True
Block 2 input: (1, 64, 128, 128)
Block 2 output: (1, 64, 128, 128)
Downsample: True
Block 3 input: (1, 64, 128, 128)
Block 3 output: (1, 64, 128, 128)
Downsample: True

--- Layer2 ---
Block 1 input: (1, 64, 128, 128)
Block 1 output: (1, 128, 64, 64)
Downsample: True
Block 2 input: (1, 128, 64, 64)
Block 2 output: (1, 128, 64, 64)
Downsample: True
Block 3 input: (1, 128, 64, 64)
Block 3 output: (1, 128, 64, 64)
Downsample: True
Block 4 input: (1, 128, 64, 64)
Block 4 output: (1, 128, 64, 64)
Downsample: True

--- Layer3 ---
Block 1 input: (1, 128, 64, 64)
Block 1 output: (1, 256, 32, 32)
Downsample: True
Block 2 input: (1, 256, 32, 32)
Block 2 output: (1, 256, 32, 32)
Downsample: True
Block 3 inpu